In [1]:
import sys
#BASE_DIR = "/home/dzigen/Desktop/ITMO/smiles2024/RAG-project-SMILES-2024-"
BASE_DIR = "/trinity/home/team06/workspace/mikhail_workspace/rag_project"
sys.path.insert(0, BASE_DIR)

from src.Reader import LLM_Model
from src.utils import ReaderMetrics, save_reader_trial_log, prepare_reader_configs, load_benchmarks_df
from src.utils import evaluate_reader

/trinity/home/team06/workspace/mikhail_workspace/mikhail_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# !!! TO CHANGE !!!
TRIAL = 3
BENCHMARKS_MAXSIZE = 500
BENCHMARKS_INFO = {'mtssquad': {'table': 'v1'}}

READER_PARAMS = {
    'prompts': {
        "assistant": "Отвечай на вопросы, используя информацию из текстов в списке ниже:",
        "system": "Ты вопросно-ответная система. Все ответы генерируй на русском языке. По вопросам отвечай кратко, чётко и конкретно. Не генерируй излишнюю информацию.",
    },
    'gen': {'max_new_tokens': 512, 'eos_token_id': 79097},
    'data_operate': {'batch_size': 8, 'num_workers':8}
    }

BERTSCORE_MODEL_PATH = "ru_electra_medium"
# !!! TO CHANGE !!!

SAVE_LOGDIR = f'./logs/trial{TRIAL}'
SAVE_HYPERPARAMS = f'{SAVE_LOGDIR}/hyperparams.json'
SAVE_READERCACHE = f'{SAVE_LOGDIR}/reader_cache.json'

In [3]:
banchmarks_path = {}
for name, version in BENCHMARKS_INFO.items():
    banchmarks_path[name] = {
        'table': f"{BASE_DIR}/data/{name}/tables/{version['table']}/benchmark.csv",
    }

In [4]:
benchmarks_df = load_benchmarks_df(banchmarks_path, BENCHMARKS_MAXSIZE)

In [5]:
reader_config = prepare_reader_configs(READER_PARAMS)
reader_metrics = ReaderMetrics(base_dir=BASE_DIR, model_path=BERTSCORE_MODEL_PATH)

Loading Meteor...
Loading ExactMatch


In [6]:
READER = LLM_Model(reader_config)

Loading checkpoint shards: 100%|██████████| 4/4 [03:30<00:00, 52.73s/it]


In [7]:
contexts = {name: [reader_config.prompts.assistant + "\n\n" + "\n\n".join([f'{i+1}. {doc}' for i, doc in enumerate(docs)]) 
                                                                      for docs in df['contexts'].to_list()] for name, df in benchmarks_df.items()}

In [8]:
reader_scores, reader_cache = evaluate_reader(benchmarks_df, READER, reader_metrics, contexts)

  0%|          | 0/500 [00:00<?, ?it/s]/trinity/home/team06/workspace/mikhail_workspace/mikhail_venv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/trinity/home/team06/workspace/mikhail_workspace/mikhail_venv/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 500/500 [21:40<00:00,  2.60s/it, BLEU2=0.367, BLEU1=0.45, ExactMatch=0.04, METEOR=0.629, BertScore=nan]     


In [9]:
save_reader_trial_log(SAVE_LOGDIR, reader_scores, SAVE_HYPERPARAMS, SAVE_READERCACHE, 
                      reader_cache, BENCHMARKS_INFO, BENCHMARKS_MAXSIZE, READER_PARAMS)